## Code permettant de réaliser les 7 entrainements


In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.losses import DiceLoss, DiceCELoss

import torch
from utilities import train
from Load import prepareTest , prepareTrain 


In [ ]:

#Contient tous les entrainement que l'on veut réaliser , de l'IRM 1 à 6 et 9 pour le mix
MRI = [ "1","2","3","4","5","6","9"]


In [ ]:
for x in MRI :
    print("This is the iteration number " + str(x ))
    
    #Crée le dossier ou sera stocké le modèle
    os.mkdir('H:\TDSI\cc359_preprocessed\Stockage\MRI' +str(x) + str(x) )
    #Dirige vers le path ou est stocké  le modèle
    model_dir = 'H:\TDSI\cc359_preprocessed\Stockage\MRI' +str(x) + str(x) 

    #Path vers l'emplcament des données déja divisées en volumes et maps de segmentation
    source_dir = 'H:\TDSI\cc359_preprocessed\MRI' + str(x)
   #preparetrain : load les données d'entrainement
    source_in = prepareTrain(source_dir, cache=True)
    cible_dir = 'H:\TDSI\cc359_preprocessed\MRI' + str(x)
    #preparetest : load les données de test
    cible_in = prepareTest(cible_dir, cache=True)

    data_in = source_in , cible_in

    
    #Choix du device sur GPU et instantiation du UNET
    device = torch.device("cuda:0")
    model = UNet(
        dimensions=3,
        in_channels=1,
        out_channels=2,
        channels=(16, 32, 64, 128, 256), 
        strides=(2, 2, 2, 2),
        num_res_units=2,
        norm=Norm.BATCH,
    ).to(device)

   
    
    #loss_function = DiceCELoss(to_onehot_y=True, sigmoid=True, squared_pred=True, ce_weight=calculate_weights(1792651250,2510860).to(device))
    loss_function = DiceLoss(to_onehot_y=True, sigmoid=True, squared_pred=True)
    # On peut changer le taux d'apprentissage , dans ce cas il est 10^-4
    optimizer = torch.optim.Adam(model.parameters(), 1e-4, weight_decay=1e-5, amsgrad=True)

    # Appel de la fonction train pour 200 epochs
    if __name__ == '__main__':
        train(model, data_in, loss_function, optimizer, 200, model_dir)

